<a href="https://colab.research.google.com/github/iris-baptista/daa_seamCarving/blob/main/G20_111812_EurisaPatricio_122701_IrisBaptista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Minitrabalho DAA - "*Seam Carving*"
# Ano Letivo 2024/2025

## 1. Bibliotecas Importadas

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

## 2. Representação de Dados

### 2.1 Como está a representar a energia dos pixels no seu grafo? Qual foi o critério para esta escolha? Que tipo de grafo representa o problema em questão?

**Optou-se por representar a energia dos pixels como peso das arestas no grafo. **

Os **nós/vértices** estarão associados aos pixels da imagem, e o **peso das arestas** corresponderão à energia de cada pixel **sucessor/destino**, que neste caso será calculada com base na combinação linear do valor da intensidade dos pixels na sua vizinhança inferior, conforme mencionado no enunciado. Contudo, sabe-se que o grafo será um digrafo, pesado e acíclico, ou seja um DAG, O que será útil para encontrarmos o caminho mais curto, ou seja, com menor energia, e, posteriormente, fazermos a sua remoção.

### 2.2 Qual é a representação computacional de grafo que está a utilizar? Por exemplo, matriz de adjacência, lista/mapa de adjacências ou uma outra alternativa?

A representação computacional do grafo que iremos utilizar é o **Mapa de Adjacências**, um dicionário onde cada pixel terá como valor outro dicionário que apontará para os seus vizinhos inferiores, sendo o respetivo valor de cada vizinho uma aresta (tendo a sua respetiva energia como peso).

### 2.3 Identifique as vantagens e desvantagens da sua representação de grafo escolhida e os critérios utilizados para a sua escolha. Por exemplo, a sua escolha facilita a implementação de alguma operação específica? Ou faz com que as operações fiquem mais eficientes (em relação ao tempo e ao espaço em memória)?

Neste problema, identificamos que o grafo será esparso. Ele terá aproximadamente W × H vértices e, em média, m ≈ 3WH arestas, pois cada vértice pode se conectar a no máximo 3 vizinhos inferiores.
Assim, a densidade do grafo será:

                             m / n² ≈ 3 / (W × H)

Como este valor é muito baixo e m « n², podemos concluir que é  mesmo um grafo esparso.
Sendo assim vantajoso usar um mapa de adjacências.


**Vantagens:**
- Melhor uso da memória (O(n + m)).
- Verificação e inserção de arestas em tempo O(1).
- Permite associar informações (como energia) diretamente às arestas.

**Desvantagens:**
- Menos eficiente em grafos densos (o que não é o caso).


### 2.4 Se optou por utilizar as classes Graph/Digraph fornecidas na Semana 6 para a sua solução, identifique também as possíveis modificações que teve de realizar nas classes.

*Responder*...

## 3. API Seam Carving

### 3.1 Estrutura de Dados

In [2]:

class Vertex:
    def __init__(self, vertex_id):
        self._vertex_id = vertex_id

    def __hash__(self):
        return hash(self._vertex_id)

    def __str__(self):
        return str(self._vertex_id)

    def vertex_id(self):
        return self._vertex_id

class Edge:
    def __init__(self, u, v, weight=0):
        self._vertex_1 = u
        self._vertex_2 = v
        self._weight = weight

    def endpoints(self):
        return (self._vertex_1, self._vertex_2)

    def cost(self):
        return self._weight

class Graph:
    def __init__(self):
        self._adjancencies = {}
        self._vertices = {}
        self._n = 0
        self._m = 0

    def insert_vertex(self, vertex_id):
        if vertex_id not in self._vertices:
            v = Vertex(vertex_id)
            self._vertices[vertex_id] = v
            self._adjancencies[v] = {}
            self._n += 1

    def insert_edge(self, u_id, v_id, weight=0):
        if u_id not in self._vertices:
            self.insert_vertex(u_id)
        if v_id not in self._vertices:
            self.insert_vertex(v_id)
        u = self._vertices[u_id]
        v = self._vertices[v_id]
        e = Edge(u, v, weight)
        self._adjancencies[u][v] = e
        self._adjancencies[v][u] = e
        self._m += 1

    def get_vertex(self, vertex_id):
        return self._vertices[vertex_id]

class DiGraph(Graph):
    def __init__(self):
        super().__init__()
        self._in_adjancencies = {}

    def insert_vertex(self, vertex_id):
        super().insert_vertex(vertex_id)
        v = self.get_vertex(vertex_id)
        self._in_adjancencies[v] = {}

    def insert_edge(self, u_id, v_id, weight=0):
        if u_id not in self._vertices:
            self.insert_vertex(u_id)
        if v_id not in self._vertices:
            self.insert_vertex(v_id)
        u = self._vertices[u_id]
        v = self._vertices[v_id]
        e = Edge(u, v, weight)
        self._adjancencies[u][v] = e
        self._in_adjancencies[v][u] = e
        self._m += 1


In [3]:

class SeamCarving:
    def __init__(self, image):
        self.image = image
        self.height, self.width = image.shape[:2]
        self.energy_map = self._calculate_energy()
        self.graph = DiGraph()





### 3.2 Calcular a energia da imagem

3.2.1 Implemente e teste a função auxiliar _calculate_energy() que irá calcular a energia da imagem atual. A função deve devolver o mapa da energia da imagem como um ndarray (numpy array).

3.2.2 Apresente uma análise da complexidade desta função em relação ao tempo e ao espaço extra de memória utilizados (em função da dimensão da imagem).

### 3.3 Encontrar a costura de menor energia (steam).

3.3.1 Descreva os passos e as modificações realizadas no grafo e/ou no algoritmo para reduzir o problema do seam carving para o problema do caminho mais curto num grafo (shortest path). Indique qual algoritmo shortest path que escolheu implementar e o motivo da sua escolha.

Resposta...

3.3.2 Implemente e teste o método find_vertical_steam()que deverá encontrar o caminho de menor energia.

3.3.3 Apresente uma análise completa da complexidade do seu algoritmo em função da dimensão da imagem, e compare-a com a complexidade de outras versões do algoritmo shortest path.

3.3.4 Analise e compare a complexidade do seu algoritmo baseado em grafos com uma solução que utiliza a estratégia de programação dinâmica. Na sua pesquisa bibliográfica e análise, deve usar fontes credíveis, tais como, artigos científicos, manuais técnicos ou recursos webgráficos devidamente acreditados.

### 3.4 Remover uma costura (steam) da imagem.

3.4.1 Implemente e teste o método remove_vertical_steam(steam) que recebe uma coleção com a sequência de pixels da costura, steam, e remove-os da imagem atual.

3.4.2 Apresente uma análise da complexidade desta operação em função da dimensão da imagem.

## 4. Validação

### 4.1 Crie uma função que receba uma imagem e um fator de escala como entrada, e que devolva a imagem redimensionada utilizando a API SeamCarving. A função deve ser capaz de lidar com o redimensionamento da largura e da altura de uma imagem, consoante a escolha do utilizador.

### 4.2 Reduza a largura da imagem img-broadway_tower.jpg para 70% da sua largura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

### 4.3 Reduza a largura da imagem img-brent-cox-unsplash.jpg para 60% da sua altura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

## 5. Questões Éticas

### a) Se colaborou com alguém fora do seu grupo, indique aqui os respetivos nomes.

Não.

### b) Deve citar todas as fontes utilizadas fora do material da UC.